# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [1]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [2]:
  import gymnasium as gym

  def run_episode(agent_function, max_steps=1000):
      """Run one episode in the LunarLander-v3 environment using the provided agent."""

      # Initialize the environment
      env = gym.make("LunarLander-v3", render_mode="human")

      # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
      observation, info = env.reset()

      # run one episode
      for _ in range(max_steps):
          # call the agent function to select an action
          action = agent_function(observation)

          print (f"Obs: {observation} -> Action: {action}")

          # step: execute an action in the environment
          observation, reward, terminated, truncated, info = env.step(action)

          env.render()

          if terminated:
              print(f"Final Reward: {reward}")
              break

      env.close()
      return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [3]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00340729  1.4115185  -0.34513122  0.02658651  0.00395493  0.07817743
  0.          0.        ] -> Action: 0
Obs: [-6.8149567e-03  1.4115392e+00 -3.4466434e-01  9.0962666e-04
  7.8180190e-03  7.7269331e-02  0.0000000e+00  0.0000000e+00] -> Action: 1
Obs: [-0.01028309  1.4109586  -0.35225877 -0.02583903  0.01319964  0.10764228
  0.          0.        ] -> Action: 1
Obs: [-0.01382837  1.4097695  -0.3619399  -0.05293341  0.02052003  0.14642149
  0.          0.        ] -> Action: 1
Obs: [-0.01746607  1.4079762  -0.37350756 -0.07986572  0.03015556  0.1927284
  0.          0.        ] -> Action: 1
Obs: [-0.02119741  1.4055799  -0.38524932 -0.10678912  0.04213946  0.23970008
  0.          0.        ] -> Action: 0
Obs: [-0.02492924  1.4025855  -0.38528463 -0.13346757  0.05412082  0.23964965
  0.          0.        ] -> Action: 1
Obs: [-0.02874317  1.3989897  -0.39555773 -0.16038299  0.06815537  0.28071693
  0.          0.        ] -> Action: 1
Obs: [-0.03264847  1.3947821  -0.4070227 

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [4]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [5]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [ 0.00287857  1.3983086   0.29155454 -0.56051415 -0.00332878 -0.06604149
  0.          0.        ] -> Action: 2
Obs: [ 0.00568275  1.3862787   0.28409284 -0.5346718  -0.00696257 -0.07268257
  0.          0.        ] -> Action: 2
Obs: [ 0.00858545  1.3751893   0.2934929  -0.49288306 -0.0101557  -0.06386806
  0.          0.        ] -> Action: 2
Obs: [ 0.01144257  1.3647187   0.2891969  -0.46539432 -0.01360708 -0.06903416
  0.          0.        ] -> Action: 2
Obs: [ 0.01446896  1.3550754   0.30536965 -0.42861852 -0.0163092  -0.05404748
  0.          0.        ] -> Action: 2
Obs: [ 0.01768894  1.3462231   0.32387334 -0.39344874 -0.01815835 -0.03698605
  0.          0.        ] -> Action: 2
Obs: [ 0.02075434  1.3381662   0.30923814 -0.35812455 -0.0208258  -0.05335408
  0.          0.        ] -> Action: 2
Obs: [ 0.02396679  1.3310732   0.32334256 -0.3152813  -0.02289461 -0.04138031
  0.          0.        ] -> Action: 2
Obs: [ 0.02715902  1.3247515   0.32151085 -0.28099716 -0.0251599

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [6]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [59]:
import numpy as np

def better_reflex_agent_v2(state):
    x, y, vx, vy, angle, angular_velocity, left_contact, right_contact = state

    # Nếu đã tiếp đất → không điều khiển nữa
    if left_contact or right_contact:
        return 0

    # --- Giữ cân bằng góc ---
    if angle > 0.1:
        return 3  # Fire right thruster
    elif angle < -0.1:
        return 1  # Fire left thruster

    # --- Giảm vận tốc ngang ---
    if vx > 0.3:
        return 1  # nghiêng trái để hãm sang phải
    elif vx < -0.3:
        return 3  # nghiêng phải để hãm sang trái

    # --- Điều chỉnh độ cao ---
    if vy < -0.5:  # rơi nhanh
        return 2  # fire main engine
    elif y < 0.2 and abs(vy) > 0.1:
        return 2  # khi gần mặt đất, hãm nhẹ

    return 0





In [60]:
def evaluate_agent(agent_func, num_episodes=100, render=False):
    env = gym.make("LunarLander-v3", render_mode="human" if render else None)
    total_rewards = []
    success_count = 0

    for episode in range(num_episodes):
        state, info = env.reset()
        done = False
        episode_reward = 0

        while not done:
            action = agent_func(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            state = next_state

        total_rewards.append(episode_reward)
        if episode_reward > 200:  # giả định "thành công" nếu reward cao
            success_count += 1

    env.close()
    avg_reward = sum(total_rewards) / len(total_rewards)
    print(f"Average reward = {avg_reward:.2f}")
    print(f"Success rate  = {success_count/num_episodes*100:.2f}%  ({success_count}/{num_episodes})")

    return avg_reward


In [61]:
print("Evaluating Better Reflex Agent v2...")
better_v2_avg = evaluate_agent(better_reflex_agent_v2, num_episodes=100)


Evaluating Better Reflex Agent v2...
Average reward = -37.96
Success rate  = 12.00%  (12/100)
